In [1]:
import numpy as np
import pandas as pd

biop_covariates = pd.read_csv('./data/biop_clinical.csv', delimiter = ',')

In [2]:
np.unique(biop_covariates['BIO'], return_counts = True)

(array(['Certolizumab', 'Other', 'abatacept', 'adalimumab', 'etanercept',
        'golimumab', 'infliximab', 'rituximab', 'tocilizumab'],
       dtype=object),
 array([ 396,   77,  179,  722, 1310,  119,   59,  613,  324]))

In [3]:
import functools

das_bins = {
    0: {0: 'Good', 1: 'Moderate', 2: 'Moderate'}, 
    1: {0: 'Moderate', 1: 'Moderate', 2: 'None'},
    2: {0: 'Moderate', 1: 'None', 2: 'None'}
}

def calc_2c_das(das_type, sjc, crp, esr):
    if das_type == 'crp':
        return np.sqrt(sjc) + 0.6 * np.log(crp + 1)
    elif das_type == 'esr':
        return np.sqrt(sjc) + 0.32 * np.log(esr)
    

def get_das_bin(das_fu, delta_das):
    f_up_bin = 1
    delta_bin = 1
    
    if das_fu <= 3.2:
        f_up_bin = 0
    elif das_fu > 5.1:
        f_up_bin = 2
        
    if delta_das > 1.2:
        delta_bin = 0
    elif delta_das <= 0.6:
        delta_bin = 2
        
    return das_bins[f_up_bin][delta_bin]

def write_eular_phenotype_file(df, das_type, non0_columns, treatment = ['adalimumab'], treatment_label = 'adalimumab'):    
    outcomes_das = []
    
    das28_idx = np.where(np.logical_and(~pd.isnull(df[f'das28{das_type}.0']), ~pd.isnull(df[f'das28{das_type}.1'])))[0]
    das28_outcomes = df.iloc[das28_idx].reset_index(drop = True)
    
    for index, row in das28_outcomes.iterrows():
        outcome_das = {
            'IID': row['sample_id']
        }
        
        das = row[f'das28{das_type}.0']
        das_fu = row[f'das28{das_type}.1']
        delta_das = das - das_fu

        outcome_das['delta_das'] = delta_das
        outcome_das['eular_bin'] = get_das_bin(das_fu, delta_das)
        
        outcome_das[f'4cdas_fu'] = das_fu
        
        outcome_das[f'2cdas_bl'] = calc_2c_das(das_type, row['das_tend.0'], row['crp.0'], row['esr.0'])
        outcome_das[f'2cdas_fu'] = calc_2c_das(das_type, row['das_tend.1'], row['crp.1'], row['esr.1'])
        
        outcomes_das.append(outcome_das)
        
    efficacy_stop_idx = np.where(np.logical_and(~pd.isnull(df[f'das28{das_type}.0']), df['efficacy_stop'] == 1))[0]
    efficacy_stop_idx = np.setdiff1d(efficacy_stop_idx, das28_idx)
    efficacy_stop_outcomes = df.iloc[efficacy_stop_idx].reset_index(drop = True)
    
    for index, row in efficacy_stop_outcomes.iterrows():
        outcome_das = {
            'IID': row['sample_id']
        }

        outcome_das['delta_das'] = 0
        outcome_das['eular_bin'] = 'None'

        outcomes_das.append(outcome_das)
        
    das28_df = pd.DataFrame(outcomes_das, index = np.arange(len(outcomes_das)))
    das28_df = df.merge(das28_df, how = 'right', left_on = 'sample_id', right_on = 'IID')
    
    das28_df = das28_df.astype({'eular_bin': 'category'})
    das28_df['class'] = das28_df['eular_bin'].cat.codes
    
    bio_idx = np.where([bio in treatment for bio in das28_df['BIO'].to_numpy()])
    # bio_idx = np.where(das28_df['BIO'] == treatment)
    das28_df = das28_df.iloc[bio_idx].reset_index(drop = True)
    
    das28_df['das_type'] = das_type
    
    idx = [pd.isnull(das28_df[x]) for x in non0_columns]
    non0_idx = np.where(np.logical_not(functools.reduce(np.logical_or, idx)))[0]
    das28_df = das28_df.iloc[non0_idx].reset_index(drop = True)
    
    print(das28_df.shape, f'samples before cleaing {das_type} df')
    
    # Drop samples that are missing all covariates
    idx = [pd.isnull(das28_df[x]) for x in ['FIRSTBIO', 'WEIGHT', 'HEIGHT', 'DISDUR', 'BIO', 'AGEONSET', 'HAQ', 'SEX', 'SERO', 'CONCURRENT_DMARD', 'HAD_A', 'HAD_D']]
    
    drop_idx = np.where(np.logical_not(functools.reduce(np.logical_and, idx)))[0]
    das28_df = das28_df.iloc[drop_idx].reset_index(drop = True)
    
    #miss_idx = [pd.isnull(das28_df[x]) for x in ['FIRSTBIO', 'WEIGHT', 'HEIGHT', 'DISDUR', 'BIO', 'AGEONSET', 'HAQ', 'SEX', 'SERO', 'CONCURRENT_DMARD', 'HAD_A', 'HAD_D']]
    #n_miss = np.count_nonzero(miss_idx, axis = 0)
    #n_miss_idx = np.where(np.logical_not(np.logical_and(pd.isnull(das28_df['SMOKE']), n_miss > 0)))[0]
    #das28_df = das28_df.iloc[n_miss_idx].reset_index(drop = True)
    
    print(das28_df.shape, f'left after cleaing {das_type} df')
    
    for col in ['FIRSTBIO', 'WEIGHT', 'HEIGHT', 'DISDUR', 'SMOKE', 'BIO', 'AGEONSET', 'HAQ', 'SEX', 'SERO', 'CONCURRENT_DMARD', 'HAD_A', 'HAD_D']:
        n_mis = (np.count_nonzero(pd.isnull(das28_df[col])) / das28_df.shape[0]) * 100
        
        print(f'{col} is missing for {n_mis:2.4f}% of samples')
        
    y_bin = das28_df['class'] < 1
    y_bin = y_bin.astype(int).to_numpy()
    das28_df['class_bin'] = y_bin
    
    y_bin = das28_df['class'] > 1
    y_bin = y_bin.astype(int).to_numpy()
    das28_df['class_poor'] = y_bin
    
    print(np.unique(y_bin, return_counts = True))
    print(np.unique(das28_df['class'], return_counts = True))
    print(np.unique(das28_df['eular_bin'], return_counts = True))
        
    das28_df.to_csv(f'./data/das28_BIOP_{das_type}_{treatment_label}_outcomes.csv', index = False)

In [4]:
write_eular_phenotype_file(biop_covariates, 'crp', ['das_tend.0', 'das_vas.0', 'das_swol.0', 'crp.0', 'BIO'], treatment = 'adalimumab', treatment_label = 'adalimumab')
write_eular_phenotype_file(biop_covariates, 'esr', ['das_tend.0', 'das_vas.0', 'das_swol.0', 'crp.0', 'BIO'], treatment = 'adalimumab', treatment_label = 'adalimumab')

(461, 51) samples before cleaing crp df
(461, 51) left after cleaing crp df
FIRSTBIO is missing for 1.3015% of samples
WEIGHT is missing for 8.0260% of samples
HEIGHT is missing for 16.4859% of samples
DISDUR is missing for 2.1692% of samples
SMOKE is missing for 39.0456% of samples
BIO is missing for 0.0000% of samples
AGEONSET is missing for 1.9523% of samples
HAQ is missing for 13.0152% of samples
SEX is missing for 0.0000% of samples
SERO is missing for 10.8460% of samples
CONCURRENT_DMARD is missing for 0.8677% of samples
HAD_A is missing for 14.3167% of samples
HAD_D is missing for 14.9675% of samples
(array([0, 1]), array([379,  82]))
(array([0, 1, 2], dtype=int8), array([194, 185,  82]))
(array(['Good', 'Moderate', 'None'], dtype=object), array([194, 185,  82]))
(366, 51) samples before cleaing esr df
(366, 51) left after cleaing esr df
FIRSTBIO is missing for 1.3661% of samples
WEIGHT is missing for 6.8306% of samples
HEIGHT is missing for 14.4809% of samples
DISDUR is missing

In [5]:
write_eular_phenotype_file(biop_covariates, 'crp', ['das_tend.0', 'das_vas.0', 'das_swol.0', 'crp.0', 'BIO'], treatment = 'etanercept', treatment_label = 'etanercept')
write_eular_phenotype_file(biop_covariates, 'esr', ['das_tend.0', 'das_vas.0', 'das_swol.0', 'crp.0', 'BIO'], treatment = 'etanercept', treatment_label = 'etanercept')

(778, 51) samples before cleaing crp df
(778, 51) left after cleaing crp df
FIRSTBIO is missing for 0.8997% of samples
WEIGHT is missing for 5.9126% of samples
HEIGHT is missing for 16.5810% of samples
DISDUR is missing for 1.2853% of samples
SMOKE is missing for 25.8355% of samples
BIO is missing for 0.0000% of samples
AGEONSET is missing for 1.2853% of samples
HAQ is missing for 12.8535% of samples
SEX is missing for 0.0000% of samples
SERO is missing for 7.1979% of samples
CONCURRENT_DMARD is missing for 1.5424% of samples
HAD_A is missing for 15.5527% of samples
HAD_D is missing for 15.6812% of samples
(array([0, 1]), array([630, 148]))
(array([0, 1, 2], dtype=int8), array([310, 320, 148]))
(array(['Good', 'Moderate', 'None'], dtype=object), array([310, 320, 148]))
(655, 51) samples before cleaing esr df
(655, 51) left after cleaing esr df
FIRSTBIO is missing for 0.9160% of samples
WEIGHT is missing for 5.0382% of samples
HEIGHT is missing for 15.2672% of samples
DISDUR is missing 

In [6]:
write_eular_phenotype_file(biop_covariates, 'crp', ['das_tend.0', 'das_vas.0', 'das_swol.0', 'crp.0', 'BIO'], treatment = ['etanercept', 'adalimumab'], treatment_label = 'eta_ada')
write_eular_phenotype_file(biop_covariates, 'esr', ['das_tend.0', 'das_vas.0', 'das_swol.0', 'crp.0', 'BIO'], treatment = ['etanercept', 'adalimumab'], treatment_label = 'eta_ada')

(1239, 51) samples before cleaing crp df
(1239, 51) left after cleaing crp df
FIRSTBIO is missing for 1.0492% of samples
WEIGHT is missing for 6.6990% of samples
HEIGHT is missing for 16.5456% of samples
DISDUR is missing for 1.6142% of samples
SMOKE is missing for 30.7506% of samples
BIO is missing for 0.0000% of samples
AGEONSET is missing for 1.5335% of samples
HAQ is missing for 12.9136% of samples
SEX is missing for 0.0000% of samples
SERO is missing for 8.5553% of samples
CONCURRENT_DMARD is missing for 1.2914% of samples
HAD_A is missing for 15.0928% of samples
HAD_D is missing for 15.4157% of samples
(array([0, 1]), array([1009,  230]))
(array([0, 1, 2], dtype=int8), array([504, 505, 230]))
(array(['Good', 'Moderate', 'None'], dtype=object), array([504, 505, 230]))
(1021, 51) samples before cleaing esr df
(1021, 51) left after cleaing esr df
FIRSTBIO is missing for 1.0774% of samples
WEIGHT is missing for 5.6807% of samples
HEIGHT is missing for 14.9853% of samples
DISDUR is mi

In [7]:
write_eular_phenotype_file(biop_covariates, 'crp', ['das_tend.0', 'das_vas.0', 'das_swol.0', 'crp.0', 'BIO'], treatment = ['etanercept', 'adalimumab', 'Certolizumab', 'golimumab', 'infliximab'], treatment_label = 'TNFi')
write_eular_phenotype_file(biop_covariates, 'esr', ['das_tend.0', 'das_vas.0', 'das_swol.0', 'crp.0', 'BIO'], treatment = ['etanercept', 'adalimumab', 'Certolizumab', 'golimumab', 'infliximab'], treatment_label = 'TNFi')

(1600, 51) samples before cleaing crp df
(1600, 51) left after cleaing crp df
FIRSTBIO is missing for 1.5625% of samples
WEIGHT is missing for 6.2500% of samples
HEIGHT is missing for 16.5625% of samples
DISDUR is missing for 1.4375% of samples
SMOKE is missing for 29.4375% of samples
BIO is missing for 0.0000% of samples
AGEONSET is missing for 1.3750% of samples
HAQ is missing for 13.3125% of samples
SEX is missing for 0.0000% of samples
SERO is missing for 9.1875% of samples
CONCURRENT_DMARD is missing for 1.0000% of samples
HAD_A is missing for 16.0000% of samples
HAD_D is missing for 15.9375% of samples
(array([0, 1]), array([1287,  313]))
(array([0, 1, 2], dtype=int8), array([642, 645, 313]))
(array(['Good', 'Moderate', 'None'], dtype=object), array([642, 645, 313]))
(1267, 51) samples before cleaing esr df
(1267, 51) left after cleaing esr df
FIRSTBIO is missing for 1.4996% of samples
WEIGHT is missing for 5.8406% of samples
HEIGHT is missing for 15.2328% of samples
DISDUR is mi

In [8]:
write_eular_phenotype_file(biop_covariates, 'crp', ['das_tend.0', 'das_vas.0', 'das_swol.0', 'crp.0', 'BIO'], treatment = ['Certolizumab', 'golimumab', 'infliximab'], treatment_label = 'cert_gol_inflix')
write_eular_phenotype_file(biop_covariates, 'esr', ['das_tend.0', 'das_vas.0', 'das_swol.0', 'crp.0', 'BIO'], treatment = ['Certolizumab', 'golimumab', 'infliximab'], treatment_label = 'cert_gol_inflix')

(361, 51) samples before cleaing crp df
(361, 51) left after cleaing crp df
FIRSTBIO is missing for 3.3241% of samples
WEIGHT is missing for 4.7091% of samples
HEIGHT is missing for 16.6205% of samples
DISDUR is missing for 0.8310% of samples
SMOKE is missing for 24.9307% of samples
BIO is missing for 0.0000% of samples
AGEONSET is missing for 0.8310% of samples
HAQ is missing for 14.6814% of samples
SEX is missing for 0.0000% of samples
SERO is missing for 11.3573% of samples
CONCURRENT_DMARD is missing for 0.0000% of samples
HAD_A is missing for 19.1136% of samples
HAD_D is missing for 17.7285% of samples
(array([0, 1]), array([278,  83]))
(array([0, 1, 2], dtype=int8), array([138, 140,  83]))
(array(['Good', 'Moderate', 'None'], dtype=object), array([138, 140,  83]))
(246, 51) samples before cleaing esr df
(246, 51) left after cleaing esr df
FIRSTBIO is missing for 3.2520% of samples
WEIGHT is missing for 6.5041% of samples
HEIGHT is missing for 16.2602% of samples
DISDUR is missing

In [9]:
write_eular_phenotype_file(biop_covariates, 'crp', ['das_tend.0', 'das_vas.0', 'das_swol.0', 'crp.0', 'BIO'], treatment = ['abatacept', 'rituximab', 'tocilizumab'], treatment_label = 'nTNFi')
write_eular_phenotype_file(biop_covariates, 'esr', ['das_tend.0', 'das_vas.0', 'das_swol.0', 'crp.0', 'BIO'], treatment = ['abatacept', 'rituximab', 'tocilizumab'], treatment_label = 'nTNFi')

(647, 51) samples before cleaing crp df
(647, 51) left after cleaing crp df
FIRSTBIO is missing for 0.6182% of samples
WEIGHT is missing for 6.0278% of samples
HEIGHT is missing for 16.5379% of samples
DISDUR is missing for 2.0093% of samples
SMOKE is missing for 8.5008% of samples
BIO is missing for 0.0000% of samples
AGEONSET is missing for 1.8547% of samples
HAQ is missing for 12.0556% of samples
SEX is missing for 0.0000% of samples
SERO is missing for 2.9366% of samples
CONCURRENT_DMARD is missing for 0.4637% of samples
HAD_A is missing for 16.5379% of samples
HAD_D is missing for 15.4560% of samples
(array([0, 1]), array([459, 188]))
(array([0, 1, 2], dtype=int8), array([186, 273, 188]))
(array(['Good', 'Moderate', 'None'], dtype=object), array([186, 273, 188]))
(558, 51) samples before cleaing esr df
(558, 51) left after cleaing esr df
FIRSTBIO is missing for 0.5376% of samples
WEIGHT is missing for 5.3763% of samples
HEIGHT is missing for 15.9498% of samples
DISDUR is missing f